In [ ]:
from glob import glob
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import shutil
import imageio
import torchvision
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import time
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:

def imshow(inp,cmap=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp,cmap)
   

## Check if GPU is present

In [ ]:
is_cuda = False
if torch.cuda.is_available():
    is_cuda = True
    

## Load data into PyTorch tensors

In [ ]:
#Load brain tumor data
simple_transform = transforms.Compose([transforms.Resize((224,224))
                                       ,transforms.ToTensor()
                                       ,transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])

train_data_dir = '/datasets/home/75/075/aakumall/Project/assets_final/train/'
val_data_dir = '/datasets/home/75/075/aakumall/Project/assets_final/val/'
test_data_dir = '/datasets/home/75/075/aakumall/Project/assets_final/test/'
train = ImageFolder(train_data_dir, simple_transform)
valid = ImageFolder(val_data_dir, simple_transform)
test = ImageFolder(test_data_dir, simple_transform)



In [ ]:
print(train.class_to_idx)
print(train.classes) 
print train.imgs[333] 
classes = ('C1', 'C2', 'C3')


In [ ]:
view_img = imageio.imread('/datasets/home/75/075/aakumall/Project/assets/train/Class1/425.jpg')


plt.imshow(np.uint8(view_img))
plt.show()

imshow(train[333][0])

## Create data generators

In [ ]:

batch_size = 64

#Code to use weighted sampler to get classes with equal probability
# class_sample_count = [708, 1426, 930] # dataset has 708 class-1 samples, 1426 class-2 samples,930 class-3 samples
# weights = 1 / torch.DoubleTensor(class_sample_count)
# sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, batch_size)

train_data_loader = torch.utils.data.DataLoader(train,batch_size=batch_size, shuffle = True)
valid_data_loader = torch.utils.data.DataLoader(valid,batch_size=32,shuffle=True)
test_data_loader = torch.utils.data.DataLoader(test,batch_size=32,num_workers=3)

## Transfer learning using VGG

In [ ]:
vgg = models.vgg16(pretrained=True)

## Freeze layers

In [ ]:
for param in vgg.features.parameters():
    param.requires_grad = False

vgg.classifier[6].out_features = 3

for layer in vgg.classifier.children():
    if(type(layer) == nn.Dropout):
        layer.p = 0.6
vgg = vgg.cuda()

In [ ]:
vgg

In [ ]:
optimizer = optim.SGD(vgg.classifier.parameters(),lr=0.0001,momentum=0.5)

In [ ]:
def fit(epoch,model,data_loader,phase='training',volatile=False):
    
    #Check for training or validation phase
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
        
    running_loss = 0.0
    running_correct = 0
    
    #Iterate over the data
    for batch_idx , (data,target) in enumerate(data_loader):
        if is_cuda:
            data,target = data.cuda(),target.cuda()
        data , target = Variable(data,volatile),Variable(target)
        if phase == 'training':
            optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output,target)
        
        running_loss += F.cross_entropy(output,target,size_average=False).data[0]
        preds = output.data.max(dim=1,keepdim=True)[1]
        
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct/len(data_loader.dataset)
    
    print('{phase} loss is {loss: 5.2} and {phase} accuracy is {running_correct}/{dlen} {accuracy:10.4}'.format(phase = phase,loss = loss, running_correct = running_correct,dlen = len((data_loader.dataset)), accuracy = accuracy))
#     print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

In [ ]:
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]
for epoch in range(1,4):
    epoch_loss, epoch_accuracy = fit(epoch,vgg,train_data_loader,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,vgg,valid_data_loader,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

In [ ]:
plt.plot(range(1,len(train_losses)+1),train_losses,'bo',label = 'training loss')
plt.plot(range(1,len(val_losses)+1),val_losses,'r',label = 'validation loss')
plt.legend()

In [ ]:
plt.plot(range(1,len(train_accuracy)+1),train_accuracy,'bo',label = 'train accuracy')
plt.plot(range(1,len(val_accuracy)+1),val_accuracy,'r',label = 'val accuracy')
plt.legend()

# Make predictions

In [ ]:
dataiter = iter(test_data_loader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))



for i in images:
    i = i.unsqueeze(0) #Insert axis for # of images - # of channels, height, width exists
#     i = Variable(i) # wrap image tensor into PyTorch variable 
images = Variable(images)

if is_cuda:
    images = images.cuda()
preds = vgg(images)

_, predicted = torch.max(preds, 1)
preds = predicted.data.cpu().numpy()
print('Predicted: ', ' '.join('%5s' % classes[preds[j]]
                             for j in range(64)))

### Adjusting dropout

### Data augmentation

## Calculating preconvoluted features


## Visualizing intermediate CNN layers

## Visualizing weights

In [ ]:
vgg = models.vgg16(pretrained=True).cuda()

In [ ]:
vgg.state_dict().keys()

In [ ]:

cnn_weights = vgg.state_dict()['features.0.weight'].cpu()

In [ ]:
fig = plt.figure(figsize=(30,30))
fig.subplots_adjust(left=0,right=1,bottom=0,top=0.8,hspace=0,wspace=0.2)
for i in range(30):
    ax = fig.add_subplot(12,6,i+1,xticks=[],yticks=[])
    imshow(cnn_weights[i])

In [ ]:
cnn_weights.shape